<a href="https://colab.research.google.com/github/4sz5sz6sz/AI-Text-Classifier-DACON2025/blob/main/improved_tfidf_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TF-IDF + XGBoost 개선 실험

**개선 사항:**
- 하이퍼파라미터 튜닝 추가
- 교차 검증 적용
- 피처 개수 조정
- 성능 비교 분석

## 🚀 Google Colab 실행 가이드

## 파일 업로드 방법

**대용량 파일(500MB+)**: Google Drive 공개 링크 사용 (아래 코드 참고)
**소용량 파일**: 직접 업로드

**주의사항:**
- 파일 업로드는 한 번만 하면 됩니다
- 런타임이 재시작되면 다시 업로드해야 합니다
- 인코딩 오류가 발생하면 자동으로 여러 방식을 시도합니다

**실행 순서:**
1. 모든 셀을 순서대로 실행
2. 파일 업로드가 요청되면 데이터 파일들을 업로드
3. 최종 결과는 자동으로 다운로드됩니다

# Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
import matplotlib.pyplot as plt

# Data Load & Split

In [ ]:
import pandas as pd
import os
from google.colab import files

# 코랩에서 파일 업로드 (처음 실행 시에만)
def upload_files_if_needed():
    required_files = ['train.csv', 'test.csv', 'sample_submission.csv']
    missing_files = [f for f in required_files if not os.path.exists(f)]
    
    if missing_files:
        print(f"다음 파일들을 업로드해주세요: {missing_files}")
        uploaded = files.upload()
        print("파일 업로드 완료!")
        return uploaded
    else:
        print("모든 필요한 파일이 이미 존재합니다.")
        return None

# 강화된 CSV 읽기 함수
def read_csv_robust(file_path):
    """다양한 방법으로 CSV 파일을 읽어보는 함수"""
    
    # 방법 1: 다양한 인코딩 시도
    encodings = ['utf-8', 'utf-8-sig', 'cp949', 'euc-kr', 'latin-1', 'iso-8859-1']
    
    for encoding in encodings:
        try:
            print(f"'{encoding}' 인코딩으로 시도 중...")
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ 성공! '{encoding}' 인코딩으로 파일을 읽었습니다.")
            return df
        except UnicodeDecodeError:
            print(f"❌ '{encoding}' 인코딩 실패")
            continue
        except Exception as e:
            print(f"❌ '{encoding}' 인코딩에서 오류: {e}")
            continue
    
    # 방법 2: 바이너리 모드로 읽어서 인코딩 감지
    try:
        print("인코딩 자동 감지를 시도합니다...")
        import chardet
        
        with open(file_path, 'rb') as f:
            raw_data = f.read()
            result = chardet.detect(raw_data)
            encoding = result['encoding']
            confidence = result['confidence']
            
        print(f"감지된 인코딩: {encoding} (신뢰도: {confidence:.2f})")
        
        if confidence > 0.7:  # 신뢰도가 70% 이상인 경우만
            df = pd.read_csv(file_path, encoding=encoding)
            print(f"✅ 자동 감지된 인코딩으로 성공!")
            return df
            
    except ImportError:
        print("chardet 라이브러리가 없습니다. 설치를 시도합니다...")
        !pip install chardet
        import chardet
        # 위 코드 재실행
        with open(file_path, 'rb') as f:
            raw_data = f.read()
            result = chardet.detect(raw_data)
            encoding = result['encoding']
            
        df = pd.read_csv(file_path, encoding=encoding)
        print(f"✅ chardet 설치 후 성공!")
        return df
    except Exception as e:
        print(f"❌ 자동 감지 실패: {e}")
    
    # 방법 3: 에러 처리 옵션 사용
    try:
        print("에러 무시 모드로 시도합니다...")
        df = pd.read_csv(file_path, encoding='utf-8', errors='ignore')
        print("✅ 에러 무시 모드로 성공!")
        return df
    except Exception as e:
        print(f"❌ 에러 무시 모드 실패: {e}")
    
    raise Exception("❌ 모든 방법으로 파일을 읽는데 실패했습니다.")

# 파일 업로드 (필요한 경우)
uploaded = upload_files_if_needed()

# 파일 읽기
try:
    print("\n=== train.csv 읽기 ===")
    train = read_csv_robust('train.csv')
    print(f"Train 데이터 shape: {train.shape}")
    print(f"Train 컬럼: {train.columns.tolist()}")

    print("\n=== test.csv 읽기 ===")
    test = read_csv_robust('test.csv')
    print(f"Test 데이터 shape: {test.shape}")
    print(f"Test 컬럼: {test.columns.tolist()}")

    print("\n=== sample_submission.csv 읽기 ===")
    sample_submission = read_csv_robust('sample_submission.csv')
    print(f"Sample submission shape: {sample_submission.shape}")
    print(f"Sample submission 컬럼: {sample_submission.columns.tolist()}")

except Exception as e:
    print(f"❌ 파일 읽기 실패: {e}")
    print("다음을 확인해주세요:")
    print("1. 파일이 올바르게 업로드되었는지")
    print("2. 파일명이 정확한지 (train.csv, test.csv, sample_submission.csv)")
    print("3. 파일이 손상되지 않았는지")

=== train.csv 읽기 ===
'utf-8-sig' 인코딩으로 시도 중...
'utf-8-sig' 인코딩에서 다른 오류: Error tokenizing data. C error: out of memory
'utf-8' 인코딩으로 시도 중...
'utf-8-sig' 인코딩에서 다른 오류: Error tokenizing data. C error: out of memory
'utf-8' 인코딩으로 시도 중...
성공! 'utf-8' 인코딩으로 파일을 읽었습니다.
Train 데이터 shape: (97172, 3)

=== test.csv 읽기 ===
'utf-8-sig' 인코딩으로 시도 중...
성공! 'utf-8-sig' 인코딩으로 파일을 읽었습니다.
Test 데이터 shape: (1962, 4)
성공! 'utf-8' 인코딩으로 파일을 읽었습니다.
Train 데이터 shape: (97172, 3)

=== test.csv 읽기 ===
'utf-8-sig' 인코딩으로 시도 중...
성공! 'utf-8-sig' 인코딩으로 파일을 읽었습니다.
Test 데이터 shape: (1962, 4)


In [ ]:
# 대용량 파일 다운로드 (Google Drive 공개 링크)
import gdown

# 파일 ID 설정 (Google Drive 링크에서 추출)
TRAIN_FILE_ID = "1teA9GmYlIsutaDLWvCCsLeh7833t-TC_"  # 제공받은 train.csv ID
TEST_FILE_ID = ""     # test.csv ID (필요시 입력)
SAMPLE_FILE_ID = ""   # sample_submission.csv ID (필요시 입력)

def download_from_drive(file_id, filename):
    if file_id:
        url = f'https://drive.google.com/uc?id={file_id}'
        gdown.download(url, filename, quiet=False)
        print(f"✅ {filename} 다운로드 완료!")

# 파일 다운로드 실행
if TRAIN_FILE_ID and not os.path.exists('train.csv'):
    print("📥 train.csv 다운로드 중...")
    download_from_drive(TRAIN_FILE_ID, 'train.csv')
    
if TEST_FILE_ID and not os.path.exists('test.csv'):
    download_from_drive(TEST_FILE_ID, 'test.csv')
    
if SAMPLE_FILE_ID and not os.path.exists('sample_submission.csv'):
    download_from_drive(SAMPLE_FILE_ID, 'sample_submission.csv')

In [ ]:
#train = pd.read_csv('./train.csv', encoding='utf-8-sig')
#test = pd.read_csv('./test.csv', encoding='utf-8-sig')

# 데이터 확인
try:
    print("=== 데이터 기본 정보 ===")
    print(f"Train shape: {train.shape}")
    print(f"Test shape: {test.shape}")
    
    # 컬럼 확인
    print(f"\nTrain 컬럼: {train.columns.tolist()}")
    print(f"Test 컬럼: {test.columns.tolist()}")
    
    # target 변수 확인
    if 'generated' in train.columns:
        print(f"\nGenerated 비율: {train['generated'].mean():.3f}")
        print(f"Generated 값 분포:")
        print(train['generated'].value_counts())
    else:
        print("\n⚠️ 'generated' 컬럼을 찾을 수 없습니다.")
        print("사용 가능한 컬럼:", train.columns.tolist())
    
    # 데이터 미리보기
    print("\n=== Train 데이터 미리보기 ===")
    print(train.head())
    
    print("\n=== Test 데이터 미리보기 ===")
    print(test.head())
    
    # 결측값 확인
    print("\n=== 결측값 확인 ===")
    print("Train 결측값:")
    print(train.isnull().sum())
    print("\nTest 결측값:")
    print(test.isnull().sum())
    
except NameError:
    print("❌ 데이터가 로드되지 않았습니다. 위의 셀을 먼저 실행해주세요.")
except Exception as e:
    print(f"❌ 데이터 확인 중 오류: {e}")

Train shape: (97172, 3)
Test shape: (1962, 4)
Generated 비율: 0.082


In [ ]:
# 데이터 전처리 및 분할
try:
    # 테스트 데이터 컬럼명 통일 (필요한 경우)
    if 'paragraph_text' in test.columns and 'full_text' not in test.columns:
        test = test.rename(columns={'paragraph_text': 'full_text'})
        print("✅ Test 데이터의 'paragraph_text' 컬럼을 'full_text'로 변경했습니다.")
    
    # 필요한 컬럼 확인
    required_train_cols = ['title', 'full_text', 'generated']
    required_test_cols = ['title', 'full_text']
    
    missing_train_cols = [col for col in required_train_cols if col not in train.columns]
    missing_test_cols = [col for col in required_test_cols if col not in test.columns]
    
    if missing_train_cols:
        print(f"❌ Train 데이터에서 누락된 컬럼: {missing_train_cols}")
        print(f"사용 가능한 컬럼: {train.columns.tolist()}")
        raise ValueError("필요한 컬럼이 누락되었습니다.")
    
    if missing_test_cols:
        print(f"❌ Test 데이터에서 누락된 컬럼: {missing_test_cols}")
        print(f"사용 가능한 컬럼: {test.columns.tolist()}")
        raise ValueError("필요한 컬럼이 누락되었습니다.")
    
    # 데이터 분할
    X = train[['title', 'full_text']]
    y = train['generated']
    
    print(f"✅ 피처 데이터 shape: {X.shape}")
    print(f"✅ 타겟 데이터 shape: {y.shape}")
    
    # Train-Validation 분할
    from sklearn.model_selection import train_test_split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42
    )
    
    print(f"✅ 훈련 세트: {X_train.shape}")
    print(f"✅ 검증 세트: {X_val.shape}")
    print(f"✅ 훈련 세트 타겟 비율: {y_train.mean():.3f}")
    print(f"✅ 검증 세트 타겟 비율: {y_val.mean():.3f}")
    
except Exception as e:
    print(f"❌ 데이터 전처리 오류: {e}")
    print("데이터 구조를 다시 확인해주세요.")

# 개선된 TF-IDF Vectorization

In [5]:
# 피처 개수 늘리고 다양한 n-gram 시도
get_title = FunctionTransformer(lambda x: x['title'], validate=False)
get_text = FunctionTransformer(lambda x: x['full_text'], validate=False)

# 메모리 효율적인 벡터화 (피처 개수 조정)
vectorizer = FeatureUnion([
    ('title', Pipeline([('selector', get_title),
                        ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=2000,
                                                min_df=3, max_df=0.9))])),
    ('full_text', Pipeline([('selector', get_text),
                            ('tfidf', TfidfVectorizer(ngram_range=(1,2), max_features=8000,
                                                    min_df=3, max_df=0.9))])),
])

# 피처 변환
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)

print(f"피처 차원: {X_train_vec.shape[1]}")

MemoryError: 

# 베이스라인 모델 (비교용)

In [ ]:
# 기본 XGBoost (베이스라인)
xgb_baseline = XGBClassifier(random_state=42)
xgb_baseline.fit(X_train_vec, y_train)

val_probs_baseline = xgb_baseline.predict_proba(X_val_vec)[:, 1]
auc_baseline = roc_auc_score(y_val, val_probs_baseline)
print(f"베이스라인 Validation AUC: {auc_baseline:.4f}")

# 하이퍼파라미터 튜닝된 모델

In [ ]:
# 개선된 하이퍼파라미터
xgb_improved = XGBClassifier(
    n_estimators=200,           # 트리 개수 증가
    max_depth=6,                # 깊이 조정
    learning_rate=0.1,          # 학습률 조정
    subsample=0.8,              # 서브샘플링
    colsample_bytree=0.8,       # 피처 서브샘플링
    reg_alpha=0.1,              # L1 정규화
    reg_lambda=1.0,             # L2 정규화
    random_state=42,
    n_jobs=-1
)

xgb_improved.fit(X_train_vec, y_train)

val_probs_improved = xgb_improved.predict_proba(X_val_vec)[:, 1]
auc_improved = roc_auc_score(y_val, val_probs_improved)
print(f"개선된 모델 Validation AUC: {auc_improved:.4f}")
print(f"성능 향상: {auc_improved - auc_baseline:.4f}")

# 교차 검증으로 안정성 확인

In [ ]:
# 5-fold 교차 검증
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 전체 훈련 데이터로 교차 검증
X_full_vec = vectorizer.fit_transform(X)
cv_scores = cross_val_score(xgb_improved, X_full_vec, y,
                           cv=cv, scoring='roc_auc', n_jobs=-1)

print(f"교차 검증 AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"개별 점수: {cv_scores}")

# 성능 비교 시각화

In [ ]:
# 모델 성능 비교
models = ['베이스라인', '개선된 모델']
scores = [auc_baseline, auc_improved]

plt.figure(figsize=(8, 6))
bars = plt.bar(models, scores, color=['lightblue', 'orange'], alpha=0.7)
plt.ylim(0.85, max(scores) + 0.01)
plt.ylabel('Validation AUC')
plt.title('모델 성능 비교')
plt.grid(True, alpha=0.3)

# 점수 표시
for bar, score in zip(bars, scores):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
             f'{score:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

# Inference (최고 성능 모델 사용)

In [ ]:
# 최종 모델로 전체 데이터 재훈련
final_model = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    n_jobs=-1
)

# 전체 훈련 데이터로 학습
X_full_vec = vectorizer.fit_transform(X)
final_model.fit(X_full_vec, y)

# 테스트 데이터 예측
test = test.rename(columns={'paragraph_text': 'full_text'})
X_test = test[['title', 'full_text']]
X_test_vec = vectorizer.transform(X_test)

probs = final_model.predict_proba(X_test_vec)[:, 1]
print(f"예측 완료. 예측값 범위: [{probs.min():.3f}, {probs.max():.3f}]")

# Submission

In [ ]:
# Submission 파일 생성
try:
    # sample_submission 파일 읽기 (안전하게)
    if 'sample_submission' not in locals():
        sample_submission = read_csv_robust('sample_submission.csv')
    
    print(f"Sample submission shape: {sample_submission.shape}")
    print(f"Sample submission 컬럼: {sample_submission.columns.tolist()}")
    
    # 예측값 할당
    sample_submission['generated'] = probs
    
    print(f"예측값 통계:")
    print(f"- 최솟값: {probs.min():.4f}")
    print(f"- 최댓값: {probs.max():.4f}")
    print(f"- 평균값: {probs.mean():.4f}")
    print(f"- 표준편차: {probs.std():.4f}")
    
    # 파일 저장
    output_filename = 'improved_submission.csv'
    sample_submission.to_csv(output_filename, index=False)
    print(f"✅ 제출 파일 저장 완료: {output_filename}")
    
    # 파일 다운로드 (코랩에서)
    try:
        from google.colab import files
        files.download(output_filename)
        print(f"✅ 파일 다운로드 시작: {output_filename}")
    except ImportError:
        print("로컬 환경에서 실행 중입니다. 파일이 현재 디렉터리에 저장되었습니다.")
    
    # 성능 개선 요약
    print(f"\n=== 🎯 성능 개선 요약 ===")
    if 'auc_baseline' in locals() and 'auc_improved' in locals():
        print(f"베이스라인 AUC: {auc_baseline:.4f}")
        print(f"개선된 AUC: {auc_improved:.4f}")
        print(f"개선폭: +{auc_improved - auc_baseline:.4f}")
    if 'cv_scores' in locals():
        print(f"교차검증 AUC: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
    
    print(f"\n🔥 최종 모델 예측 완료!")
    
except Exception as e:
    print(f"❌ 제출 파일 생성 오류: {e}")
    print("이전 단계들이 모두 성공적으로 실행되었는지 확인해주세요.")